<a href="https://colab.research.google.com/github/firdaaacy/Tensorflow-Exercise/blob/main/image-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================================================
# PROBLEM A2 
#
# Build a Neural Network Model for Horse or Human Dataset.
# The test will expect it to classify binary classes. 
# Your input layer should accept 150x150 with 3 bytes color as the input shape.
# Don't use lambda layers in your model.
#
# The dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
#
# Desired accuracy and validation_accuracy > 83%
# ======================================================================================

import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

def solution_A2():
    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')

    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()

    TRAINING_DIR = 'data/horse-or-human'
    VALIDATION_DIR = 'data/validation-horse-or-human'

    train_datagen = ImageDataGenerator(rescale=1 / 255,
                                      # width_shift_range=0.2,
                                      # height_shift_range=0.2,
                                      #  horizontal_flip=True,
                                      #  zoom_range=0.25,
                                      #  shear_range=0.25,
                                      #  rotation_range=10,
                                      #  fill_mode='nearest'
                                       )

    train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=100,
                                                        class_mode='binary',
                                                        shuffle=True,
                                                        interpolation="bilinear",
                                                        target_size=(150, 150))

    validation_datagen = ImageDataGenerator(rescale=1/255) 
    validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                             batch_size=100,
                                                             class_mode='binary',
                                                             shuffle=True,
                                                             interpolation="bilinear",
                                                             target_size=(150, 150))

    model = tf.keras.models.Sequential(
        [tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(150, 150, 3)),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(rate=0.3),
         tf.keras.layers.Dense(128, activation='relu'),
         
         tf.keras.layers.Dense(1, activation='sigmoid')
         ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_generator, validation_data=validation_generator, epochs=10)
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    model = solution_A2()
    model.save("model_A2.h5")


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/10
11/11 [==============================] - 109s 10s/step - loss: 0.6909 - accuracy: 0.5239 - val_loss: 0.6842 - val_accuracy: 0.5000
Epoch 2/10
11/11 [==============================] - 103s 9s/step - loss: 0.6634 - accuracy: 0.6037 - val_loss: 0.5987 - val_accuracy: 0.8359
Epoch 3/10
11/11 [==============================] - 104s 9s/step - loss: 0.5784 - accuracy: 0.7176 - val_loss: 0.4315 - val_accuracy: 0.8359
Epoch 4/10
11/11 [==============================] - 100s 9s/step - loss: 0.5057 - accuracy: 0.7449 - val_loss: 0.3397 - val_accuracy: 0.8672
Epoch 5/10
11/11 [==============================] - 103s 10s/step - loss: 0.3896 - accuracy: 0.8238 - val_loss: 0.2833 - val_accuracy: 0.8945
Epoch 6/10
11/11 [==============================] - 98s 9s/step - loss: 0.3114 - accuracy: 0.8539 - val_loss: 0.3178 - val_accuracy: 0.9062
Epoch 7/10
11/11 [==============================] - 99s 9s/step - loss

In [1]:
# ======================================================================================================
# PROBLEM A3 
#
# Build a classifier for the Human or Horse Dataset with Transfer Learning. 
# The test will expect it to classify binary classes.
# Note that all the layers in the pre-trained model are non-trainable.
# Do not use lambda layers in your model.
#
# The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
# Inception_v3, pre-trained model used in this problem is developed by Google.
#
# Desired accuracy and validation_accuracy > 93%.
# =======================================================================================================

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

def solution_A3():
    inceptionv3='https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
    local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)
    pre_trained_model.load_weights(local_weights_file)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    # last_layer = pre_trained_model.get_layer('mixed6')
    last_output = pre_trained_model.output

    # data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    # urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    # local_file = 'horse-or-human.zip'
    # zip_ref = zipfile.ZipFile(local_file, 'r')
    # zip_ref.extractall('data/horse-or-human')

    # data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    # urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    # local_file = 'validation-horse-or-human.zip'
    # zip_ref = zipfile.ZipFile(local_file, 'r')
    # zip_ref.extractall('data/validation-horse-or-human')
    # zip_ref.close()

    _TRAIN_URL = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
    _TEST_URL = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
    urllib.request.urlretrieve(_TRAIN_URL, 'horse-or-human.zip')
    local_zip = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/horse-or-human/')
    zip_ref.close()
    urllib.request.urlretrieve(_TEST_URL, 'testdata.zip')
    local_zip = 'testdata.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/testdata/')
    zip_ref.close()

    x = layers.Flatten()(last_output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)
    callbacks = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10)

    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])

    train_dir = 'tmp/horse-or-human/'
    validation_dir = 'tmp/testdata/'

    train_datagen = ImageDataGenerator(rescale =1/255,
                                       fill_mode = 'nearest',
                                       horizontal_flip = True,
                                       rotation_range = 45,
                                       width_shift_range=0.25,
                                       height_shift_range=0.25,
                                       shear_range=0.25,
                                       zoom_range=0.25)
    
    train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                        batch_size=100,
                                                        class_mode='binary',
                                                        shuffle=True,
                                                        interpolation='bilinear',
                                                        target_size=(150, 150))
    
    validation_datagen = ImageDataGenerator(rescale=1/255) 
    validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                             batch_size=100,
                                                             class_mode='binary',
                                                             shuffle=True,
                                                             interpolation='bilinear',
                                                             target_size=(150, 150))
    

    model.fit(train_generator, validation_data=validation_generator, epochs=5, callbacks=[callbacks])
    
    # model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # model.fit(train_x, train_y, validation_data=(x_test, y_test), epochs=10, callbacks=[callbacks])

    return model

# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    model = solution_A3()
    model.save("model_A3.h5")


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/5
11/11 [==============================] - 70s 6s/step - loss: 1.9462 - acc: 0.7478 - val_loss: 0.0376 - val_acc: 0.9961
Epoch 2/5
11/11 [==============================] - 62s 6s/step - loss: 0.3452 - acc: 0.8793 - val_loss: 0.0542 - val_acc: 1.0000
Epoch 3/5
11/11 [==============================] - 62s 6s/step - loss: 0.1878 - acc: 0.9357 - val_loss: 0.0305 - val_acc: 1.0000
Epoch 4/5
11/11 [==============================] - 62s 6s/step - loss: 0.1064 - acc: 0.9659 - val_loss: 0.0153 - val_acc: 0.9961
Epoch 5/5
11/11 [==============================] - 62s 6s/step - loss: 0.1657 - acc: 0.9416 - val_loss: 0.0135 - val_acc: 1.0000


SUBMISSION B


In [ ]:
# ========================================================================================
# PROBLEM B3
#
# Build a CNN based classifier for Rock-Paper-Scissors dataset.
# Your input layer should accept 150x150 with 3 bytes color as the input shape.
# This is unlabeled data, use ImageDataGenerator to automatically label it.
# Don't use lambda layers in your model.
#
# The dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
#
# Desired accuracy AND validation_accuracy > 83%
# ========================================================================================

import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator


# class MyCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('val_accuracy') > 0.86 and logs.get('accuracy') > 0.83):
#             print("\nAccuracy and Validation Accuracy reached 83%, so cancelled training!")
#             self.model.stop_training = True


def solution_B3():
    data_url = 'https://github.com/dicodingacademy/assets/releases/download/release-rps/rps.zip'
    urllib.request.urlretrieve(data_url, 'rps.zip')
    local_file = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/')
    zip_ref.close()

    TRAINING_DIR = "data/rps/"
    training_datagen = ImageDataGenerator(validation_split=0.2,
                                          rescale=1 / 255,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          rotation_range=10,
                                          fill_mode='nearest',
                                          horizontal_flip=True
                                          )

    train_generator = training_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                           batch_size=32,
                                                           class_mode="categorical",
                                                           subset="training",
                                                           shuffle=True,
                                                           target_size=(150, 150))

    validation_generator = training_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                                batch_size=32,
                                                                class_mode="categorical",
                                                                subset="validation",
                                                                target_size=(150, 150))

    model = tf.keras.models.Sequential(
        [tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),
         tf.keras.layers.Dropout(0.3),
         
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(64, activation="relu"),
         tf.keras.layers.Dropout(0.2),
         tf.keras.layers.Dense(3, activation="softmax")
         ])

    # callbacks = MyCallback()
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(train_generator, validation_data=validation_generator, epochs=10)
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    model = solution_B3()
    model.save("model_B3.h5")

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Epoch 1/10
63/63 [==============================] - 94s 1s/step - loss: 0.8611 - accuracy: 0.5962 - val_loss: 0.5359 - val_accuracy: 0.7004
Epoch 2/10
63/63 [==============================] - 82s 1s/step - loss: 0.1472 - accuracy: 0.9544 - val_loss: 0.3060 - val_accuracy: 0.8492
Epoch 3/10
63/63 [==============================] - 81s 1s/step - loss: 0.0555 - accuracy: 0.9846 - val_loss: 0.4073 - val_accuracy: 0.8294
Epoch 4/10
63/63 [==============================] - 81s 1s/step - loss: 0.0645 - accuracy: 0.9777 - val_loss: 0.3579 - val_accuracy: 0.7996
Epoch 5/10
63/63 [==============================] - 80s 1s/step - loss: 0.0381 - accuracy: 0.9891 - val_loss: 0.2494 - val_accuracy: 0.9107
Epoch 6/10
63/63 [==============================] - 81s 1s/step - loss: 0.0386 - accuracy: 0.9881 - val_loss: 0.1950 - val_accuracy: 0.9187
Epoch 7/10
63/63 [==============================] - 81s 1s/step - loss: 0.029

PAKET C


In [ ]:
# =======================================================================================================
# PROBLEM C3
#
# Build a CNN based classifier for Cats vs Dogs dataset.
# Your input layer should accept 150x150 with 3 bytes color as the input shape.
# This is unlabeled data, use ImageDataGenerator to automatically label it.
# Don't use lambda layers in your model.
#
# The dataset used in this problem is originally published in https://www.kaggle.com/c/dogs-vs-cats/data
#
# Desired accuracy and validation_accuracy > 72%
# ========================================================================================================

import tensorflow as tf
import urllib.request
import zipfile
import tensorflow as tf
import os
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# class MyCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('val_accuracy') > 0.73 and logs.get('accuracy') > 0.73):
#             print("\nAccuracy and Validation Accuracy reached 73%, so cancelled training!")
#             self.model.stop_training = True


def solution_C3():
    data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/cats_and_dogs.zip'
    urllib.request.urlretrieve(data_url, 'cats_and_dogs.zip')
    local_file = 'cats_and_dogs.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/')
    zip_ref.close()

    BASE_DIR = 'data/cats_and_dogs_filtered'
    train_dir = os.path.join(BASE_DIR, 'train')
    validation_dir = os.path.join(BASE_DIR, 'validation')

    train_datagen = ImageDataGenerator(rescale=1 / 255,
                                       fill_mode='nearest',
                                       horizontal_flip=True,
                                       rotation_range=45,
                                       width_shift_range=0.25,
                                       height_shift_range=0.25,
                                       shear_range=0.2,
                                       zoom_range=0.2

                                       )  # YOUR CODE HERE

    val_datagen = ImageDataGenerator(
        rescale=1 / 255.0
    )

    # YOUR IMAGE SIZE SHOULD BE 150x150
    # Make sure you used "categorical"
    train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                        batch_size=100,
                                                        class_mode='binary',
                                                        shuffle=True,
                                                        target_size=(150, 150))

    val_generator = train_datagen.flow_from_directory(directory=validation_dir,
                                                      batch_size=100,
                                                      class_mode='binary',
                                                      shuffle=True,
                                                      target_size=(150, 150)

                                                      )  # YOUR CODE HERE

    # model = tf.keras.models.Sequential([
    #     # YOUR CODE HERE, end with a Neuron Dense, activated by 'sigmoid'
    #     tf.keras.layers.Dense(1, activation='sigmoid')
    # ])

    model = tf.keras.models.Sequential(
        [tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),

         tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(2, 2),

        #  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        #  tf.keras.layers.MaxPooling2D(2, 2),

        #  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        #  tf.keras.layers.MaxPooling2D(2, 2),
        #  tf.keras.layers.Dropout(0.3),

         tf.keras.layers.Flatten(),
         # tf.keras.layers.Dense(128, activation='relu'),
         tf.keras.layers.Dense(64, activation='relu'),
        #  tf.keras.layers.Dropout(0.3),
         # # YOUR CODE HERE, end with 3 Neuron Dense, activated by softmax
         tf.keras.layers.Dense(1, activation='sigmoid')
         ])

    # callbacks = MyCallback()
    model.compile(optimizer=RMSprop(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(train_generator, validation_data=val_generator, epochs=30)
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model = solution_C3()
    model.save("model_C3.h5")


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
20/20 [==============================] - 126s 6s/step - loss: 0.8640 - accuracy: 0.5055 - val_loss: 0.6904 - val_accuracy: 0.5560
Epoch 2/30
20/20 [==============================] - 107s 5s/step - loss: 0.6960 - accuracy: 0.5235 - val_loss: 0.6958 - val_accuracy: 0.5000
Epoch 3/30
20/20 [==============================] - 106s 5s/step - loss: 0.6908 - accuracy: 0.5390 - val_loss: 0.8869 - val_accuracy: 0.5230
Epoch 4/30
20/20 [==============================] - 105s 5s/step - loss: 0.6972 - accuracy: 0.5755 - val_loss: 0.6581 - val_accuracy: 0.6310
Epoch 5/30
20/20 [==============================] - 104s 5s/step - loss: 0.6614 - accuracy: 0.5920 - val_loss: 0.6820 - val_accuracy: 0.5620
Epoch 6/30
20/20 [==============================] - 106s 5s/step - loss: 0.6706 - accuracy: 0.6050 - val_loss: 0.6963 - val_accuracy: 0.5400
Epoch 7/30
20/20 [==============================] - 106s 5s/step - los